In [3]:
import dicttoxml
import numpy as np
import pandas as pd

from Question import Question, Category, Options
from XMLdoc import XMLdoc, Module, Task, Document, HIT
from createSurvey import Survey

from xml.dom.minidom import parseString


In [108]:
sample_responses = pd.read_csv("sample_responses.tsv", "\t", encoding = "utf-8")
sample_responses

,hitid,taskid,workerid,module,varname,response
0,5,0,Mm,intro,intro,I have read and understood the task instructio...
1,5,1,Mm,screen0,screen_valid0,0
2,5,1,Mm,screen0,screen_label0,1
3,5,2,Mm,screen1,screen_valid1,0
4,5,2,Mm,screen1,screen_label1,0
...,...,...,...,...,...,...
367,2,50,ling2,label,label,NaN
368,2,53,ling2,label,valid,No
369,2,53,ling2,label,label,Junk
370,2,55,ling2,label,valid,Yes


In [9]:
np.unique(sample_responses["taskid"])

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
       51, 52, 53, 54, 55, 56], dtype=int64)

In [33]:
valid_inds = np.where(sample_responses["varname"] == "valid")[0]
label_inds = np.where(sample_responses["varname"] == "label")[0]

valids = sample_responses["response"][valid_inds]
valids_task = sample_responses["taskid"][valid_inds]
labels = sample_responses["response"][label_inds]
labels_task = sample_responses["taskid"][label_inds]

tasks = np.unique(sample_responses["taskid"][qidx])

In [34]:
labels, labels_task

(13          NaN
 15     Personal
 17          NaN
 19          NaN
 21          NaN
          ...   
 363        Junk
 365         NaN
 367         NaN
 369        Junk
 371         NaN
 Name: response, Length: 150, dtype: object, 13      8
 15     10
 17     11
 19     13
 21     15
        ..
 363    48
 365    49
 367    50
 369    53
 371    55
 Name: taskid, Length: 150, dtype: int64)

In [35]:
valids, valids_task

(12     Yes
 14      No
 16     Yes
 18     Yes
 20     Yes
       ... 
 362     No
 364    Yes
 366    Yes
 368     No
 370    Yes
 Name: response, Length: 150, dtype: object, 12      8
 14     10
 16     11
 18     13
 20     15
        ..
 362    48
 364    49
 366    50
 368    53
 370    55
 Name: taskid, Length: 150, dtype: int64)

In [166]:
answer = []
agreement = []

for i in range(len(tasks)):
    task = tasks[i]
    valid_res = valids.iloc[np.where(valids_task == task)[0]]
    [ans, count] = np.unique(valid_res, return_counts = True)
    #answer.append(ans[0])
    #agreement.append(count[0]/len(valid_res))
    label_res = labels.iloc[np.where(labels_task == task)[0]]
    [ans, count] = np.unique([str(x) for x in label_res], return_counts = True)
    answer.append(ans[0])
    agreement.append(count[0]/len(label_res))
    
answer = np.array(answer)
agreement = [round(x, 2) for x in agreement]
answer[np.where(answer == 'nan')[0]] = "Valid"
answer = [str(x) for x in answer]

In [167]:
answer

['Valid',
 'Advertisement',
 'Other',
 'Advertisement',
 'Personal',
 'Valid',
 'Junk',
 'Other',
 'Valid',
 'Junk',
 'Other',
 'Junk',
 'Junk',
 'Other',
 'Junk',
 'Junk',
 'Advertisement',
 'Valid',
 'Personal',
 'Valid',
 'Multimedia News',
 'Advertisement',
 'Valid',
 'Other',
 'Valid',
 'Valid',
 'Junk',
 'Junk',
 'Valid',
 'Valid',
 'Advertisement',
 'Other',
 'Advertisement',
 'Advertisement',
 'Other',
 'Valid',
 'Advertisement',
 'Junk',
 'Other',
 'Other',
 'Personal',
 'Advertisement',
 'Junk',
 'Other',
 'Other',
 'Valid',
 'Valid',
 'Junk',
 'Multimedia News',
 'Other']

In [168]:
agreed

array([], dtype=int64)

In [169]:
agreement[7]

0.67

In [156]:
answer[agreed]

TypeError: only integer scalar arrays can be converted to a scalar index

In [157]:
agreed = np.where(agreement == 1.0)[0]
tasks[agreed]

array([], dtype=int64)

In [158]:
tasks[-agreed]

array([], dtype=int64)

In [159]:
tasks

array([ 6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22,
       23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39,
       40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55],
      dtype=int64)

In [177]:
input_csv = pd.read_csv("mturk_scrapes_sample.csv", encoding = "utf-8", index_col=0)
input_csv["task"] = [i + 6 for i in range(len(input_csv))]

In [178]:
input_csv

,id,text,date,article_prior,snippet,snip_sent_inds,html,task
0,41772,Novotech Awarded Asia-Pacific Biotech CRO of t...,60119,1,"SYDNEY, May 31, 2019 - (ACN Newswire) - Novote...","[4, 5, 6, 7, 8, 9, 10, 11]","<p>SYDNEY, May 31, 2019 - (ACN Newswire) - Nov...",6
1,63900,Get lunch from the best restaurants in your ci...,60119,-1,Get lunch from the best restaurants in your ci...,"[0, 1, 2]",<p>Get lunch from the best restaurants in your...,7
2,88400,Reviews ### Review: 2019 13-inch MacBook Pro -...,60119,1,Review: AirUnleashed is a great wireless charg...,"[6, 7, 8, 9, 10]",<p>Review: AirUnleashed is a great wireless ch...,8
3,79744,The editorial staff also publishes a daily on...,60119,-1,The editorial staff also publishes a daily onl...,"[0, 1]",<p>The editorial staff also publishes a daily ...,9
4,92089,Press J to jump to the feed. Press question ma...,60119,-1,Press J to jump to the feed. Press question ma...,"[0, 1, 2, 3, 4, 5, 6, 7]",<p>Press J to jump to the feed. Press question...,10
5,31392,"UK volunteers still honor US airmen, 75 years ...",60119,1,""" This is kind of the twilight years for them,...","[63, 64, 65, 66, 67, 68, 69, 70]","<p>"" This is kind of the twilight years for th...",11
6,65786,FacebookEmail Twitter Google+ LinkedIn Pintere...,60119,1,Tiger Woods sounds off on Hank Haney SiriusXM ...,"[6, 7, 8, 9, 10, 11]",<p>Tiger Woods sounds off on Hank Haney Sirius...,12
7,4387,Andy Ruiz 20lbs heavier than Anthony Joshua ##...,60119,-1,Katie Taylor 134.6 vs. Delfine Persoon 130.6##...,"[8, 9, 10, 11, 12, 13, 14]",<p>Katie Taylor 134.6 vs. Delfine Persoon 130....,13
8,37996,"Thank you for Reading! ### Please log in, or s...",60119,1,To civil liberties lawyers suing to block the ...,"[23, 24, 25, 26, 27, 28]",<p>To civil liberties lawyers suing to block t...,14
9,9130,PREFERRED HOME IMPROVEMENT ### Published June ...,60119,-1,PREFERRED HOME IMPROVEMENT###Published June 01...,"[0, 1]",<p>PREFERRED HOME IMPROVEMENT</p><p>Published ...,15


In [180]:
for i in range(len(tasks)):
    task = tasks[i]
    row = np.where(input_csv["task"] == task)[0]
    input_csv.loc[row, "answer"]= answer[i]
    input_csv.loc[row, "agreement"] = agreement[i]

In [183]:
input_csv.to_csv("mturk_scrapes_sample.csv", encoding = "utf-8")
input_csv

,id,text,date,article_prior,snippet,snip_sent_inds,html,task,answer,agreement
0,41772,Novotech Awarded Asia-Pacific Biotech CRO of t...,60119,1,"SYDNEY, May 31, 2019 - (ACN Newswire) - Novote...","[4, 5, 6, 7, 8, 9, 10, 11]","<p>SYDNEY, May 31, 2019 - (ACN Newswire) - Nov...",6,Valid,1.00
1,63900,Get lunch from the best restaurants in your ci...,60119,-1,Get lunch from the best restaurants in your ci...,"[0, 1, 2]",<p>Get lunch from the best restaurants in your...,7,Advertisement,1.00
2,88400,Reviews ### Review: 2019 13-inch MacBook Pro -...,60119,1,Review: AirUnleashed is a great wireless charg...,"[6, 7, 8, 9, 10]",<p>Review: AirUnleashed is a great wireless ch...,8,Other,0.67
3,79744,The editorial staff also publishes a daily on...,60119,-1,The editorial staff also publishes a daily onl...,"[0, 1]",<p>The editorial staff also publishes a daily ...,9,Advertisement,1.00
4,92089,Press J to jump to the feed. Press question ma...,60119,-1,Press J to jump to the feed. Press question ma...,"[0, 1, 2, 3, 4, 5, 6, 7]",<p>Press J to jump to the feed. Press question...,10,Personal,1.00
5,31392,"UK volunteers still honor US airmen, 75 years ...",60119,1,""" This is kind of the twilight years for them,...","[63, 64, 65, 66, 67, 68, 69, 70]","<p>"" This is kind of the twilight years for th...",11,Valid,1.00
6,65786,FacebookEmail Twitter Google+ LinkedIn Pintere...,60119,1,Tiger Woods sounds off on Hank Haney SiriusXM ...,"[6, 7, 8, 9, 10, 11]",<p>Tiger Woods sounds off on Hank Haney Sirius...,12,Junk,1.00
7,4387,Andy Ruiz 20lbs heavier than Anthony Joshua ##...,60119,-1,Katie Taylor 134.6 vs. Delfine Persoon 130.6##...,"[8, 9, 10, 11, 12, 13, 14]",<p>Katie Taylor 134.6 vs. Delfine Persoon 130....,13,Other,0.67
8,37996,"Thank you for Reading! ### Please log in, or s...",60119,1,To civil liberties lawyers suing to block the ...,"[23, 24, 25, 26, 27, 28]",<p>To civil liberties lawyers suing to block t...,14,Valid,1.00
9,9130,PREFERRED HOME IMPROVEMENT ### Published June ...,60119,-1,PREFERRED HOME IMPROVEMENT###Published June 01...,"[0, 1]",<p>PREFERRED HOME IMPROVEMENT</p><p>Published ...,15,Junk,0.67


In [186]:
input_csv[input_csv["agreement"] >= 1].to_csv("mturk_scrapes_sample_KEY.csv")

In [13]:
import re
illegal = re.compile(u"(?<![\uD800-\uDBFF])[\uDC00-\uDFFF]|[\uD800-\uDBFF](?![\uDC00-\uDFFF])|[\x00-\x08\x0B\x0C\x0E-\x1F\x7F-\x9F\uFEFF\uFFFE\uFFFF]")

with open("temp.txt") as f:
    test = f.read()
    print("" in test)
    cleaned = illegal.sub("\uFFFD", test)
    print("" in cleaned)

True
False


In [14]:
cleaned

'<?xml version="1.0" encoding="UTF-8" ?><xml><modules><module><name>intro</name><header>Read the instructions carefully.</header><questions><question><varname>intro</varname><questiontext>Please take time to read the instructions carefully before you proceed.</questiontext><valuetype>categorical</valuetype><content><categories><category><text>I have read and understood the task instructions.</text><value>I have read and understood the task instructions.</value></category></categories></content></question></questions></module><module><name>example</name><header>Examples</header><questions><question><varname>example</varname><questiontext>Please take time to understand why the example label is appropriate before you proceed.</questiontext><valuetype>categorical</valuetype><content><categories><category><text>I have read and understand the label choice for this example.</text><value>I have read and understand the label choice for this example.</value></category></categories></content></qu